In [2]:
import numpy as np
import open3d as o3d
import threading

In [3]:
def add_draw(geometries, window_name, width, height):
    vis = o3d.visualization.O3DVisualizer(title=window_name,
                                          width=width,
                                          height=height)
    count = 0
    for geometry in geometries:
        vis.add_geometry(f"geometry_{count}", geometry)
        count += 1
    vis.reset_camera_to_default()
    o3d.visualization.gui.Application.instance.add_window(vis)

In [ ]:
app = o3d.visualization.gui.Application.instance
app.initialize()
app.run()

In [ ]:
pc_rad = 1.0
r = 0.4
big_bbox = o3d.geometry.AxisAlignedBoundingBox((-pc_rad, -3, -pc_rad),
                                               (6.0 + r, 1.0 + r, pc_rad))
add_draw([big_bbox], "Open3D empty_box", 640, 480)

In [ ]:
visualizer = o3d.WebVisualizer(window_uid="window_0")
visualizer.show()